# Steps For buiding Seq2Seq Chatbot

## 1. Data Preprocessing

1. Import Dataset : Movie Dailogue
2. Creating Dictionary
3. Creating a list of all the conversations
4. Getting separatetely the questions and answers
5. Cleaning Text
6. Cleaning Questions
7. Cleaning Answers
8. Creating a dictionary that maps each word to its number occurences
9. Creating two dictionaries that map the questions words and the answers words to a unique integer
10. Adding the last tokens to these two dictionaries tokens = 'PAD', 'EOS', 'OUT', 'SOS'
11. Trnaslating all the questions and the answers into integers and Replacing all the words that were filtered out by 'OUT'  
12. Sorting questions and answers by the length of questions because it will speedup the training and help to reduce

****** Final Output after cleaning dataset: sorted_clean_questions and sorted_clean_answers ********

In [1]:
#Import library
import tensorflow as tf
import numpy as np
import re
import time
from io import open

In [2]:
#Import Dataset
lines = open('movie_lines.txt', encoding ='utf-8', errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding ='utf-8', errors='ignore').read().split('\n')

In [3]:
print len(lines)
print len(conversations)

304714
83098


In [4]:
#Creating Dictionary
id2line={}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [5]:
#Creating a list of all the conversations
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = map(str, conversation.split(' +++$+++ '))[-1][1:-1].replace("'", "").replace(" ", "").split(",")
    conversations_ids.append(_conversation)

In [6]:
conversations[0].split(' +++$+++ ')[-1][1:-1].replace("''", "").replace(" ", "")

u"'L194','L195','L196','L197'"

In [7]:
map(str, conversations[0].split(' +++$+++ '))[-1][1:-1].replace("'", "").replace(" ", "").split(",")

['L194', 'L195', 'L196', 'L197']

In [8]:
conversations_ids[:4]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206']]

In [9]:
# Getting separatetely the questions and answers
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])
        

In [10]:
print questions[0]
print answers[0]

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.


In [11]:
#Cleaning Text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"\.", " . ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"/", " / ", text)
    text = re.sub(r",", " , ", text)
    text = re.sub(r'"', ' " ', text)
    text = re.sub(r"-", " - ", text)

    text = re.sub(r"[-<>{}+=|?'()\:@]", "", text)
    return text.replace('\n', '')

In [12]:
#Cleaning Questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    

In [13]:
clean_questions[:2]

[u'can we make this quick    roxanne korrine and andrew barrett are having an incredibly horrendous public break   up on the quad .   again . ',
 u'well ,  i thought we would start with pronunciation ,  if that is okay with you . ']

In [14]:
#Cleaning Answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [15]:
clean_answers[:2]

[u'well ,  i thought we would start with pronunciation ,  if that is okay with you . ',
 u'not the hacking and gagging and spitting part .   please . ']

In [16]:
# Creating a dictionary that maps each word to its number occurences
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [17]:
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [18]:
# Creating two dictionaries that map the questions words and the answers words to a unique integer
threshold = 20 #Hyperparameter
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1        

In [19]:
questionswords2int

{u'yellow': 2122,
 u'four': 4313,
 u'gag': 5767,
 u'woods': 0,
 u'hanging': 1,
 u'woody': 2,
 u'marching': 4322,
 u'looking': 5028,
 u'electricity': 2123,
 u'fingernails': 2124,
 u'lord': 6448,
 u'shaving': 6449,
 u'dela': 2125,
 u'callin': 4316,
 u'dell': 2126,
 u'foul': 4312,
 u'fur': 3634,
 u'stabbed': 4317,
 u'screaming': 4,
 u'disturb': 4318,
 u'prize': 2127,
 u'wooden': 5,
 u'wednesday': 6,
 u'solid': 7898,
 u'oooh': 2128,
 u'straight': 4662,
 u'tired': 6451,
 u'miller': 2130,
 u'007': 890,
 u'pulse': 6453,
 u'tires': 6454,
 u'elegant': 6455,
 u'rusty': 4358,
 u'valiant': 4319,
 u'melvin': 4320,
 u'thunder': 4321,
 u'cooking': 7,
 u'fingers': 6457,
 u'admiral': 4323,
 u'specialist': 6458,
 u'hero': 6459,
 u'reporter': 6460,
 u'herb': 6461,
 u'leaning': 2131,
 u'here': 6463,
 u'reported': 6464,
 u'china': 8,
 u'punks': 5031,
 u'cult': 4325,
 u'herr': 6466,
 u'tatum': 6467,
 u'dork': 2132,
 u'kids': 9,
 u'talby': 10,
 u'k': 91,
 u'climbed': 11,
 u'reports': 6469,
 u'military': 2134

In [22]:
threshold = 20 #Hyperparameter
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1        

In [23]:
answerswords2int

{u'yellow': 2122,
 u'four': 4313,
 u'gag': 5767,
 u'woods': 0,
 u'hanging': 1,
 u'woody': 2,
 u'marching': 4322,
 u'looking': 5028,
 u'electricity': 2123,
 u'fingernails': 2124,
 u'lord': 6448,
 u'shaving': 6449,
 u'dela': 2125,
 u'callin': 4316,
 u'dell': 2126,
 u'foul': 4312,
 u'fur': 3634,
 u'stabbed': 4317,
 u'screaming': 4,
 u'disturb': 4318,
 u'prize': 2127,
 u'wooden': 5,
 u'wednesday': 6,
 u'solid': 7898,
 u'oooh': 2128,
 u'straight': 4662,
 u'tired': 6451,
 u'miller': 2130,
 u'007': 890,
 u'pulse': 6453,
 u'tires': 6454,
 u'elegant': 6455,
 u'rusty': 4358,
 u'valiant': 4319,
 u'melvin': 4320,
 u'thunder': 4321,
 u'cooking': 7,
 u'fingers': 6457,
 u'admiral': 4323,
 u'specialist': 6458,
 u'hero': 6459,
 u'reporter': 6460,
 u'herb': 6461,
 u'leaning': 2131,
 u'here': 6463,
 u'reported': 6464,
 u'china': 8,
 u'punks': 5031,
 u'cult': 4325,
 u'herr': 6466,
 u'tatum': 6467,
 u'dork': 2132,
 u'kids': 9,
 u'talby': 10,
 u'k': 91,
 u'climbed': 11,
 u'reports': 6469,
 u'military': 2134

In [24]:
#Adding the last tokens to these two dictionaries
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

In [25]:
for token in tokens:
    questionswords2int[token] = len(questionswords2int)+1
    answerswords2int[token] = len(answerswords2int)+1

In [26]:
questionswords2word = {w_i:w for w, w_i in questionswords2int.items()}
anwerswords2word = {w_i:w for w, w_i in answerswords2int.items()}

In [28]:
# questionswords2word

In [29]:
#Adding the end of String 
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [30]:
clean_answers[:2]

[u'well ,  i thought we would start with pronunciation ,  if that is okay with you .  <EOS>',
 u'not the hacking and gagging and spitting part .   please .  <EOS>']

In [31]:
clean_questions = map(str, clean_questions)

In [32]:
#Trnaslating all the questions and the answers into integers and Replacing all the words that were filtered out by <OUT>
questions_to_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_to_int.append(ints)

In [33]:
#Trnaslating all the questions and the answers into integers and Replacing all the words that were filtered out by <OUT>
answers_to_int = []
for answers in clean_answers:
    ints = []
    for word in answers.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_to_int.append(ints)

In [34]:
# Sorting questions and answers by the length of questions
# because it will speedup the training and help to reduce the loss reason for this is will reduce the amount of padding
sorted_clean_questions = []
sorted_clean_answers = []

for length in range(1,25+1):
    for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])

In [36]:
# sorted_clean_questions

In [54]:
# answers_to_int[2]

 ## Steps For Seq2Seq model:
### 1. Creating placeholders for the inputs and tthe targets
### 2. Preprocessing the tagrgets: Adding 'SOS' token 
### 3. Creating the Encoder RNN Layer
    parameters:
        1. rnn_inputs
        2. rnn_size : number of units in the RNN layer
        3. num_layers : number of RNN layers that the model uses
        4. keep_prop
        5. sequence_length : vector of lengths 
    steps:
        1. lstm : BasicLSTMCell
        2. lstm_dropout: DropoutWrapper
        3. encoder_cell: MultiRNNCell
        4. encoder_state: bidirectional_dynamic_rnn
    output:
        1. encoder_state: internal states from the RNN layer(s)
### 4. decode_training_set
    parameters:
        1. encoder_state : states created by the encoder part of the seq2seq network
        2. decoder_state : 
        3. decoder_embedded_input
        4. sequence_length
        5. decoding_scope
        6. output_function
        7. keep_prop
        8. batch_size
    steps:
        1. attention_states 
        2. attention_keys, attention_values,attention_score_function,attention_construct_function:prepare_attention
        3. training_decoder_function: seq2seq.attention_decoder_fn_train
        4. decoder_output, decoder_final_state, decoder_final_context_state = seq2seq.dynamic_rnn_decoder
        5. decoder_output_dropout: tf.nn.dropout
        6. output_function(decoder_output_dropout)
    output:
        ****output_function(decoder_output_dropout)****
### 5. decode_test_set
### 6. Creating the Decoder RNN
    prameter:
        1. decoder_embedded_input, 
        2. decoder_embeddings_matrix, 
        3. encoder_state, 
        4. num_words,
        5. sequence_length, 
        6. num_layes,
        7. word2int,
        8. keep_prop,
        9. batch_size,
        10. rnn_size
     steps:
         1. lstm : BasicLSTMCell
         2. lstm_dropout: DropoutWrapper
         3. decoder_cell: MultiRNNCell
         4. weights: truncated_normal_initializer
         5. biases: zeros_initializer
         6. output_function: fully_connected 
         Creating Dense (Fully Connected) layer at the end of the Decoder -  
         used for generating probabilities for each word in the vocabulary
         
         7. training_predictions: decode_training_set
         8. test_predictions: decode_test_set
     output:
         1. training_predictions
         2. test_predictions
 
### 7. Building the seq2seq model
    parameter:
        1. inputs
        2. targets
        3. keep_prob
        4. batch_size
        5. sequence_length
        6. answers_num_words
        7. questions_num_words
        8. encoder_embedding_size
        9. decoder_embedding_size
        10. rnn_size 
        11. num_layers
        12. questionswords2int
     steps:
         1. encoder_embedded_input: embed_sequence
         2. encoder_state: encoder_rnn
         3. preprocessed_targets: preprocess_targets
         4. decoder_embeddings_matrix
         5. decoder_embedded_input
         6. training_predictions, test_predictions: decoder_rnn
      output:
          1. training_predictions
          2. test_predictions
         
         
     
     
     
  
    
    

In [37]:
# Creating placeholders for the inputs and tthe targets
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prop')
    return inputs, targets, lr, keep_prob

In [43]:
#Preprocessing the tagrgets
def preprocess_targets(target, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0],[batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

In [87]:
# Creating the Encoder RNN Layer
#rnn_inputs: model inputs
# rnn_size = number of inputs
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prop, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, keep_prop)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]*num_layers)
    _, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                       cell_bw= encoder_cell,
                                                       sequence_length = sequence_length,
                                                       inputs = rnn_inputs,
                                                       dtype = tf.float32)
    return encoder_state

In [88]:
def decode_training_set(encoder_state, decoder_state, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prop, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option='bahdanau', num_units=decoder_cell.output_size) 
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], 
                                                                              attention_keys,
                                                                             attention_values,
                                                                             attention_score_function,
                                                                             attention_construct_function,
                                                                             name="attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                             training_decoder_function,
                                                                                                             decoder_embedded_input,
                                                                                                             sequence_length,
                                                                                                             scope=decoding_scope) 
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prop)
    return output_function(decoder_output_dropout)

In [89]:
#Decoding the test/validation set
def decode_test_set(encoder_state,
                    decoder_state, 
                    decoder_embedded_matrix, 
                    sequence_length, 
                    decoding_scope, 
                    output_function, 
                    keep_prop, 
                    batch_size, 
                    sos_id, 
                    eos_id, 
                    maximum_length, 
                    num_words):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, 
                                                                                                                                    attention_option='bahdanau', 
                                                                                                                                    num_units=decoder_cell.output_size) 
    
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                            encoder_sate[0], 
                                                                            attention_keys,
                                                                            attention_values,
                                                                            attention_score_function,
                                                                            attention_construct_function,
                                                                            decoder_embedded_matrix,
                                                                            sos_id,
                                                                            eos_id,
                                                                            maximium_length,
                                                                            nu,_words,
                                                                            name="attn_dec_inf")
    
    test_predicitons, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                             test_decoder_function,
                                                                                                             decoder_embedded_input,
                                                                                                             sequence_length,
                                                                                                             scope=decoding_scope) 
    return test_predicitons

In [90]:
#Creating the Decoder RNN
def decoder_rnn(decoder_embedded_input, 
                decoder_embeddings_matrix, 
                encoder_state, 
                num_words,
                sequence_length, 
                num_layes,
                word2int,
                keep_prop,
                batch_size,
                rnn_size
               ):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions
 
        

In [91]:
# Building the seq2seq model
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions


In [92]:
#PART 3

#Setting the Hyperparameter
epochs = 100
batch_size = 64
num_layers = 3
rnn_size = 512
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5

In [93]:
#Defining a Session
tf.reset_default_graph()
session = tf.InteractiveSession()

In [94]:
#Load Model Input
inputs, targets, lr, keep_prob = model_inputs()

In [95]:
#Setting Sequence length to maximum length
sequence_length = tf.placeholder_with_default(25, None, name="sequence_length")

In [96]:
#Getting the shape of the inputs tensor
input_shape = tf.shape(inputs)

In [97]:
input_shape

<tf.Tensor 'Shape:0' shape=(2,) dtype=int32>

In [98]:
#Getting the training and test predictions
training_predicitons, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]), 
                                                       targets, 
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int
                                                      )

ValueError: Attempt to reuse RNNCell <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell object at 0x12cc61f10> with a different variable scope than its first use.  First use of cell was with scope 'bidirectional_rnn/fw/multi_rnn_cell/cell_0/basic_lstm_cell', this attempt is with scope 'bidirectional_rnn/fw/multi_rnn_cell/cell_1/basic_lstm_cell'.  Please create a new instance of the cell if you would like it to use a different set of weights.  If before you were using: MultiRNNCell([BasicLSTMCell(...)] * num_layers), change to: MultiRNNCell([BasicLSTMCell(...) for _ in range(num_layers)]).  If before you were using the same cell instance as both the forward and reverse cell of a bidirectional RNN, simply create two instances (one for forward, one for reverse).  In May 2017, we will start transitioning this cell's behavior to use existing stored weights, if any, when it is called with scope=None (which can lead to silent model degradation, so this error will remain until

In [ ]:
#Setting up the Loss Error, the Optimizer and Gradient Clipping
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predicitons,
                                                 targets,
                                                 tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5.,5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

In [101]:
#Padding the sequence with the <PAD> token
# pad so that each sentence has asame length
def apply_padding(bactch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequences in bactch_of_sequences])
    return [sequence + [word2int['<PAD>']]*(max_sequence_length - len(sequence)) for sequence in bactch_of_sequences]

In [ ]:
#Splitting the data nto batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers